>Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies
of the Software, and to permit persons to whom the Software is furnished to do
so, subject to the following conditions:
>
>The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
>
>THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

##### The code in this notebook leverages the [Azure Blob Inventory Report](https://learn.microsoft.com/en-us/azure/storage/blobs/blob-inventory) to create a hierarchical path structure for roll-up calculations used in analytics and reporting. 

____________________________________
*Please clean-up the managed DBFS delta tables when analysis is complete, or modify the code to use external locations.*

In [0]:
import pyspark.sql.functions as F
import pandas as pd
from typing import Iterator
from pyspark.sql.types import StringType, LongType

In [0]:
%sql DROP TABLE IF EXISTS adls_inventory_load

In [0]:
%sql DROP TABLE IF EXISTS adls_inventory_temp

In [0]:
%sql DROP TABLE IF EXISTS adls_inventory_processed

In [0]:
storage_account = '[Storage Account Name]'
container = '[Storage Account Container]'
folder_path = '[File.parquet]'

path = "abfss://{}@{}.dfs.core.windows.net/{}".format(container,storage_account,folder_path)

displayHTML('<h3>Load Path </br></br> ' + path + '</h3>')

In [0]:


df = spark.read.format('parquet').load(path)

df.write.format('delta').saveAsTable("adls_inventory_load")

In [0]:
small_file_mb = 100
small_file_bytes = small_file_mb * (1000**2)

In [0]:
df = spark.read.table('adls_inventory_load')

tmp_df = df.select(F.col('Name'),F.col('Content-Length'))
tmp_df = tmp_df.withColumn('IsDelta',F.when((F.col('Name').contains('_delta_log')==True),F.lit(1)).otherwise(F.lit(0)) )
tmp_df = tmp_df.withColumn('IsDir',F.when((F.col('Content-Length') >0),F.lit(0)).otherwise(F.lit(1)))
tmp_df = tmp_df.withColumn('IsDir',F.when((F.col('Name').contains('.parquet')==True) ,F.lit(0)).otherwise(F.col('IsDir')))
tmp_df = tmp_df.withColumn('Folder_Array',F.split(F.col('Name'),'/'))
tmp_df = tmp_df.withColumn('Parent_Array',F.array_except(F.col('Folder_Array'),F.array(F.col('Folder_Array')[F.size(F.col('Folder_Array'))-1])))
tmp_df = tmp_df.withColumn('Level',F.size(F.col('Parent_Array')))
tmp_df = tmp_df.withColumn('FileCount',F.when(F.col('IsDir')==0,F.lit(1)).otherwise(F.lit(0)))
tmp_df = tmp_df.withColumn('Content-Length-Mean',F.col('Content-Length'))
tmp_df = tmp_df.withColumn('Content-Length-Min',F.when(F.col('Content-Length')>0,F.col('Content-Length')).otherwise(F.lit(-1)) )
tmp_df = tmp_df.withColumn('SmallFile',F.when(((F.col('IsDir')==0)&(F.col('Content-Length')<=small_file_bytes)),F.lit(1)).otherwise(F.lit(0)) )




In [0]:
tmp_df2 = tmp_df.groupby('Parent_Array').agg(F.max(F.col('IsDelta')).alias('IsDelta'))
tmp_df2 = tmp_df2.withColumnRenamed('Parent_Array','Parent_Array_agg')                                 
  
inner_tmp_df = tmp_df.select('Name','Folder_Array','Parent_Array','Content-Length','Level','FileCount','Content-Length-Mean','Content-Length-Min','IsDir','SmallFile').join(tmp_df2,(tmp_df.Folder_Array == tmp_df2.Parent_Array_agg),'left')

tmp_df = inner_tmp_df.select('Name','Folder_Array','Parent_Array','Content-Length','Level','FileCount','Content-Length-Mean','Content-Length-Min','IsDir','SmallFile','IsDelta').filter(df['Name'].contains('_delta_log')==False) ## Do not count files in _delta_log

tmp_df = tmp_df.withColumn('FileType',F.when((F.col('IsDir')==0),F.split(F.col('Folder_Array')[F.size(F.col('Folder_Array'))-1],'\.')).otherwise(F.array()))
tmp_df = tmp_df.withColumn('FileType',F.when((F.size('FileType')>0),F.col('FileType')[F.size(F.col('FileType'))-1]).otherwise(F.lit('Directory')) )


tmp_df2 = tmp_df.groupby('Parent_Array').agg(F.max(F.col('FileType')).alias('FileType'))
tmp_df2 = tmp_df2.withColumnRenamed('Parent_Array','Parent_Array_agg')   

inner_tmp_df = tmp_df.select('Name','Folder_Array','Parent_Array','Content-Length','Level','FileCount','Content-Length-Mean','Content-Length-Min','IsDir','SmallFile','IsDelta').join(tmp_df2,(tmp_df.Folder_Array == tmp_df2.Parent_Array_agg),'left')

tmp_df = inner_tmp_df.select('Name','Folder_Array','Parent_Array','Content-Length','Level','FileCount','Content-Length-Mean','Content-Length-Min','IsDir','SmallFile','IsDelta','FileType')
tmp_df = tmp_df.withColumn('FileType',F.when(F.col('IsDelta')==1,F.lit('Delta')).otherwise(F.col('FileType')))



tmp_df.write.format('delta').saveAsTable("adls_inventory_temp")

In [0]:
tmp_df = spark.read.table('adls_inventory_temp')

df_list = []

levels = tmp_df.filter(F.col('Level')>0).select('Level').distinct().orderBy(F.desc('Level')).rdd.flatMap(lambda x: x).collect()

for level in levels:
  print(level)
  agg_df = tmp_df.filter((F.col('Level')==level) & (F.col('Content-Length-Min')>0)).groupby('Parent_Array').agg(F.sum('Content-Length').alias('Content-Length-Agg'),
                                                                                                                F.sum('FileCount').alias('FileCount-Agg'),
                                                                                                                F.mean('Content-Length-Mean').alias('Content-Length-Mean-Agg'),
                                                                                                                F.min('Content-Length-Min').alias('Content-Length-Min-Agg'),
                                                                                                                F.sum('SmallFile').alias('SmallFileCount'))                                                                                                        
  
  agg_df = agg_df.withColumnRenamed('Parent_Array','Parent_Array_agg')

  inner_tmp_df = tmp_df.select('Name','Folder_Array','Parent_Array','Content-Length','Level','FileCount','Content-Length-Mean','Content-Length-Min','IsDir','SmallFile','FileType').join(agg_df,(tmp_df.Folder_Array == agg_df.Parent_Array_agg),'left')
  
  inner_tmp_df =inner_tmp_df.withColumn('Content-Length',F.when(F.col('Content-Length-Agg') >0,F.col('Content-Length-Agg')).otherwise(F.col('Content-Length')))
  inner_tmp_df =inner_tmp_df.withColumn('Content-Length-Mean',F.when(F.col('Content-Length-Mean-Agg') >0,F.col('Content-Length-Mean-Agg')).otherwise(F.col('Content-Length-Mean')))
  inner_tmp_df =inner_tmp_df.withColumn('Content-Length-Min',F.when(F.col('Content-Length-Min-Agg') >0,F.col('Content-Length-Min-Agg')).otherwise(F.col('Content-Length-Min')))
  inner_tmp_df = inner_tmp_df.withColumn('FileCount',F.when(F.col('FileCount-Agg') >0,F.col('FileCount-Agg')).otherwise(F.col('FileCount')))
  inner_tmp_df =inner_tmp_df.withColumn('SmallFile',F.when(F.col('SmallFileCount') >0,F.col('SmallFileCount')).otherwise(F.col('SmallFile')))
  
  tmp_df = inner_tmp_df.select('Name','Folder_Array','Parent_Array','Content-Length','Content-Length-Mean','Content-Length-Min' ,'FileCount','SmallFile', 'Level','IsDir','FileType')
 


In [0]:
KB = float(1024)
MB = float(KB ** 2) 
MB

In [0]:
final_df = tmp_df.filter((F.col('Content-Length')>0) & (F.col('FileCount')>0) & (F.col('IsDir')==1) )
final_df.cache()
final_df = final_df.select(F.col('Name'),F.col('Folder_Array').alias('FolderPath'),F.col('FileCount'),F.col('SmallFile').alias('SmallFileCount'),(F.col('SmallFile')/F.col('FileCount')).alias('PctSmallFiles'),#F.col('Content-Length'),
                                                                                (F.col('Content-Length')/MB).alias('SizeMB'),
                                                                              #F.col('Content-Length-Mean'),
                                                                             (F.col('Content-Length-Mean')/MB).alias('SizeMeanMB'),
                                                                              #F.col('Content-Length-Min'),
                                                                              (F.col('Content-Length-Min')/MB).alias('SizeMinMB'),F.col('Level'),F.col('FileType')).orderBy('Name')

#display(final_df.orderBy('Name'))
final_df.write.format('delta').saveAsTable("adls_inventory_processed")

In [0]:
%sql
SELECT * FROM adls_inventory_processed